In [1]:
import pandas as pd
import plotly.express as px
import scipy.stats as stats

In [2]:
df1 = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\01_Data.csv') 
print(df1)
df1.head(2)

       Index  Member_ID Sales_Type Contract_Type Channel    Datetime  Term  \
0          1   66758234         렌탈          일반계약    영업방판  2019-05-06    60   
1          2   66755948         렌탈          교체계약    영업방판  2020-02-20    60   
2          3   66756657         렌탈          일반계약  홈쇼핑/방송  2019-02-28    60   
3          4   66423450        멤버십        멤버십3유형     재계약  2019-05-13    12   
4          5   66423204        멤버십        멤버십3유형     재계약  2019-05-10    12   
...      ...        ...        ...           ...     ...         ...   ...   
51296  51298   66579515         렌탈        프로모션계약   대형마트A  2019-03-01    60   
51297  51299   66799558         렌탈          일반계약   대형마트A  2019-04-01    60   
51298  51300   66799197         렌탈        프로모션계약    영업방판  2019-04-01    39   
51299  51301   66792778         렌탈          일반계약  홈쇼핑/방송  2020-02-06    60   
51300  51302   66799607         렌탈          일반계약  홈쇼핑/방송  2019-04-24    60   

      Payment_Type Product_Type  Amount_Month Customer_Type   A

,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,Address2,State,Overdue_count,Overdue_Type,Gender,Credit_Rank,Bank
0,1,66758234,렌탈,일반계약,영업방판,2019-05-06,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,9.0,새마을금고
1,2,66755948,렌탈,교체계약,영업방판,2020-02-20,60,카드이체,DES-1,102900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,2.0,현대카드


# 특성 공학 (Feature Engineering)

- **특성공학** : 학습의 목적에 맞게 데이터를 다듬거나 학습 알고리즘을 통제하는 작업
- 대표적인 특성공학 기법 :
  - Scaling & Encoding : 숫자데이터의 스케일을 맞추거나(Scaling) 문자데이터를 숫자로 변환(Encoding)
  - Imputation : 결측값을 대치하여, 새로운 데이터가 들어올 때 결측값이 있더라도 예측/분류
  - Cross Validation : 학습 데이터를 여러단계로 나누어 분할하여 학습
  - Hyper Parameter Tuning : 알고리즘 내 수학적 구조나, 학습이 수행되며 발생하는 구조를 통제
  - Imbanalnced Data Sampling (분류) : 데이터의 비율이 깨져 있는 경우, 한쪽의 데이터를 지우거나 생성하여 데이터의 비율을 임의로 맞춤
  - Feature Selection : 목포변수Y에 가장 영향령이 있는(회귀계수가 높은) 인자를 사용자가 지정한 만큼 선택해 사용하는 기법

In [30]:
cond1 = df1['State'].isin(['계약확정','기간만료'])
df1.loc[cond1, 'Target'] = '정상'
df1.loc[~cond1, 'Target'] = '해약'

In [31]:
# 변수선언
X = df1[['Age','Gender','Product_Type', 'Amount_Month','Term', 'Credit_Rank']]
Y = df1['Target']

- Pipe Line : 데이터의 전처리 과정과 모델의 학습 단계를 하나의 파이썬 객체로 묶어 처리

- Scaling
  - Standard Scaling : 평균이 0 / 표준편차 1 (-> 선형회귀 알고리즘)
  - Min Max Scaling : 최대값 1 / 최소값 0 (-> 문자데이터 / 비정형데이터)
  - Robust Scaling : 중앙값 0 / IQR(사분범위) 1 (-> 이상치가 많은 데이터)
- Encoding
  - Label Encoding : 각 문자 항목을 하나의 정수로 Mapping 처리
  - One Hot Encoding : 각 문자 항목을 여러 Column으로 변환하여, 데이터가 있으면 1 없으면 0 처리

In [32]:
from sklearn.pipeline import make_pipeline # 특성공학 + 학습수행
from sklearn.impute import SimpleImputer # 결측값을 처리 (단순대치)
from sklearn.preprocessing import StandardScaler # 스케일링 기법 (평균0 / 표준편차1)
from sklearn.preprocessing import OneHotEncoder # 인코딩 기법 적용
from sklearn.tree import DecisionTreeClassifier # 분류 학습을 수행
from sklearn.model_selection import train_test_split # 학습데이터와 검증데이터 분할

In [33]:
# random_stat = 특정 고유 정수값에 대해, 데이터가 Random하게 똑같이 추출됨
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234)

In [34]:
# 숫자 처리 파이프 구성 ( 결측값 -> 중앙값 / Scaling -> Standard)
numeric_pipe = make_pipeline(SimpleImputer(strategy='median'), StandardScaler())

In [35]:
# 문자 처리 파이프 구성 (결측값 -> 최빈값 / Encoding -> One Hot Encoding)
category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

In [36]:
# 파이프 라인을 병렬구조로 배치
from sklearn.compose import make_column_transformer

In [43]:
# 각 데이터 타입을 구분짓는 리스트 생성
numeric_list = ['Age','Amount_Month','Term','Credit_Rank']
category_list = ['Gender','Product_Type']

In [44]:
pipe_model = make_column_transformer((numeric_pipe,numeric_list),(category_pipe,category_list))

In [45]:
# 특성공학에서 학습까지 파이프라인을 구성
pipe_model2 = make_pipeline(pipe_model, DecisionTreeClassifier())

In [46]:
pipe_model2.fit(X_train, Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Amount_Month',
                                                   'Term', 'Credit_Rank']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['Gender',
                                                   'Product_Type'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [47]:
# 생성한 수식(모델) 파일형테로 변환
import pickle # 파이썬에서 선언된 객체를 변수로 저장

In [27]:
pickle.dump(pipe_model2, open('model.sav','wb'))